In [ ]:
import numpy as np
import cv2
import mahotas
import joblib
import math
import pickle

# Load the trained model and input image
MODEL = 'rfclassifier_600.sav'
BOVW = "bovw_codebook_600.pickle"

# Hu Moments
def fd_hu_moments(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    return feature

# Haralick Texture
def fd_haralick(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    haralick = mahotas.features.haralick(gray).mean(axis=0)
    return haralick

# Color Histogram
def fd_histogram(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    bins = 8
    hist = cv2.calcHist([image], [0, 1, 2], None, [bins, bins, bins], [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

# SIFT Bag of Visual Words
def feature_extract(im):
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    sift = cv2.xfeatures2d.SIFT_create()
    feature = bowDiction.compute(gray, sift.detect(gray))
    return feature.squeeze()

# Load the trained model and BOVW codebook
loaded_model = joblib.load(MODEL)

# Class-label dictionary and colours
label = {0:"10", 1:"20", 2:"50", 3:"100", 4:"200", 5:"500", 6:"2000"}
colours = [(0, 63, 123), (47, 255, 173), (238, 244, 21), (220, 126, 181), (29, 170, 255), (133, 142, 146), (255, 0, 255)]

# Load the BOVW codebook
pickle_in = open(BOVW, "rb")
dictionary = pickle.load(pickle_in)

# Initialize SIFT BOW image descriptor extractor
sift2 = cv2.xfeatures2d.SIFT_create()
bowDiction = cv2.BOWImgDescriptorExtractor(sift2, cv2.BFMatcher(cv2.NORM_L2))
bowDiction.setVocabulary(dictionary)

# Function to extract features from the image
def features(image):
    Humo = fd_hu_moments(image)
    Harl = fd_haralick(image)
    Hist = fd_histogram(image)
    Bovw = feature_extract(image)
    mfeature = np.hstack([Humo, Harl, Hist, Bovw])
    return mfeature

# Open the video capture (webcam)
cap = cv2.VideoCapture(0)  # Use '0' for the default webcam

# Check if the webcam opened successfully
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # Check if frame was read successfully
    if not ret:
        print("Error: Failed to capture frame.")
        break
    
    # Pre-process the frame (convert to grayscale, adaptive threshold, etc.)
    imgray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    thresh = cv2.adaptiveThreshold(imgray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 7, 7)
    
    # Find contours of the binary image
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Iterate through contours
    for cnt in contours:
        # Filter contours based on area to ignore small objects
        if cv2.contourArea(cnt) > 500:  # You can adjust this threshold
            # Get bounding box for contour
            rect = cv2.minAreaRect(cnt)
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            
            # Draw the bounding box around the detected contour
            cv2.drawContours(frame, [box], 0, (0, 255, 0), 2)
            
            # Crop the region of interest (ROI)
            (x, y), (w, h), _ = rect
            roi = frame[int(y - h/2):int(y + h/2), int(x - w/2):int(x + w/2)]
            if roi.size == 0:
                continue
            
            # Resize the cropped image to a standard size for prediction
            img_size = 320
            resize_ratio = 1.0 * (img_size / max(w, h))
            target_size = (int(resize_ratio * w), int(resize_ratio * h))
            resized_roi = cv2.resize(roi, target_size)
            
            # Extract features from the cropped and resized ROI
            feature_vector = features(resized_roi)
            
            # Predict the denomination using the trained model
            prediction = loaded_model.predict([feature_vector])
            predicted_label = label[prediction[0]]
            
            # Draw the predicted label on the frame
            cv2.putText(frame, predicted_label, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    
    # Display the resulting frame
    cv2.imshow("Currency Note Prediction", frame)
    
    # Break the loop when 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close windows
cap.release()
cv2.destroyAllWindows()


C:\Users\prave\AppData\Local\Temp\ipykernel_23600\752846593.py:95: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:    0.0s finished
C:\Users\prave\AppData\Local\Temp\ipykernel_23600\752846593.py:95: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 200 out of 200 | elapsed:    0.0s finished
C:\Users\prave\AppData\Local\Temp\ipykernel_23600\752846593.py:95: DeprecationWarning: `np.int0`

AttributeError: 'NoneType' object has no attribute 'squeeze'

: 